# RoBERTa Resume Experience Classifier

This notebook fine-tunes **RoBERTa-base** to predict **experience_level** (junior/mid/senior) from resume data.

**Approach:**
- Concatenates all resume columns into a single text sequence
- Fine-tunes `roberta-base` for 3-class classification
- Uses HuggingFace Trainer with stratified train/test split
- Evaluates with accuracy, F1-score, and confusion matrix


In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Check if running on CPU or CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


# Load Data

Load the cleaned resumes CSV and inspect the data distribution.


In [ ]:
TARGET_COL = "experience_level"

df = pd.read_csv("cleaned_resumes.csv", engine="python")

print("Shape:", df.shape)
print("Columns:", list(df.columns))
print("\nTarget distribution:\n", df[TARGET_COL].value_counts(dropna=False))

# Prepare Text Data

Concatenate all resume columns (except the target label) into a single text string per row. Each column is prefixed with its name in brackets for context.


In [ ]:
def clean_value(v):
    """Convert any cell to a clean string."""
    if pd.isna(v):
        return ""
    s = str(v)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def row_to_text(row, target_col):
    parts = []
    for col, val in row.items():
        if col == target_col:
            continue
        s = clean_value(val)
        if s:
            parts.append(f"[{col}] {s}")
    return " ".join(parts)

df["text"] = df.apply(lambda r: row_to_text(r, TARGET_COL), axis=1)

print(df["text"].iloc[0][:600])
print("\nAverage text length (chars):", int(df["text"].str.len().mean()))


# Create Label Mappings

Map the target labels (junior, mid, senior) to integer IDs for classification.


In [ ]:
labels = sorted(df[TARGET_COL].dropna().unique().tolist())
label2id = {l: i for i, l in enumerate(labels)}
id2label = {i: l for l, i in label2id.items()}

df["label"] = df[TARGET_COL].map(label2id)

print("Labels:", labels)


# Train/Test Split

Create stratified 80/20 train/test split to ensure balanced class distribution in both sets.


In [ ]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

print("Train:", train_df.shape, "Test:", test_df.shape)


# Tokenization

Load the RoBERTa tokenizer and tokenize the train/test text data. Text is truncated/padded to max 512 tokens.


In [ ]:
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

MAX_LEN = 512

def tokenize(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=MAX_LEN
    )

train_enc = tokenize(train_df["text"].tolist())
test_enc = tokenize(test_df["text"].tolist())


# Create PyTorch Datasets

Wrap the tokenized data and labels into PyTorch Dataset objects for the Trainer.


In [ ]:
class ResumeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = ResumeDataset(train_enc, train_df["label"].tolist())
test_dataset = ResumeDataset(test_enc, test_df["label"].tolist())


# Load Pre-trained Model

Load RoBERTa with a classification head for 3 classes (junior/mid/senior).


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)


# Training Setup

Define metrics computation, training arguments, and initialize the Trainer.


In [ ]:
def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro")
    }


In [ ]:
args = TrainingArguments(
    output_dir="roberta_resume_level",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",

    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=5,

    logging_steps=50,
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


# Evaluation & Results

Run predictions on the test set and display accuracy, F1-score, classification report, and confusion matrix.


In [ ]:
pred = trainer.predict(test_dataset)

y_true = test_df["label"].to_numpy()
y_pred = np.argmax(pred.predictions, axis=1)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Macro-F1:", f1_score(y_true, y_pred, average="macro"))

print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=labels))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

conf_mat = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix - RoBERTa")
plt.show()
